<img src="images/header.png">

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-30-events.csv', '/workspace/home/event_data/2018-11-20-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-08-events.csv', '/workspace/home/event_data/2018-11-25-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-18-events.csv', '/workspace/home/event_data/2018-11-28-events.csv', '/workspace/home/event_data/2018-11-05-events.csv', '/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-12-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-24-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Query1:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Data Modeling
1. The combination of `Session_id` and `Item_In_Session` is unique for each song played in a session. hence, they form the a nice **composite primary key** .
2. Since `Item_In_Session` can have repeated values across different sessions, `Session_id` is chosen as the **partition key** to ensure even distribution of data across nodes.
3. The required fields from the above query **artist name, song title, and song length** can be queried using the following table structure:

```sql
CREATE TABLE IF NOT EXISTS song_played (
    Session_id INT,
    Item_In_Session INT,
    Artist TEXT,
    Song TEXT,
    Length FLOAT,
    PRIMARY KEY (Session_id, Item_In_Session)
);

```


In [8]:
# Structuring the tables based on the below query
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
session.execute("DROP TABLE IF EXISTS song_played") # Drop table if it already exist to avoid duplicates


try:
    session.execute("""
     CREATE TABLE IF NOT EXISTS song_played (
        Session_id INT,
        Item_In_Session INT,
        Artist TEXT,
        Song TEXT,
        Length FLOAT,
        PRIMARY KEY (Session_id, Item_In_Session)
                   )
""")

except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv' 

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements into the `query` variable
        query = "INSERT INTO song_played(Session_id, Item_In_Session, Artist, Song, Length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query,(int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Doing a SELECT to verify that the data have been inserted into each table correctly

In [10]:
query = "select  Artist, Song, Length FROM song_played WHERE Session_id=338 AND Item_In_Session=4"
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows) # Convert to DataFrame

except Exception as e:
    print(e)

display(df)    


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2:

### 2. Give me only the following: name of artist, song (sorted by itemInSession), and user (first and last name) for userid = 10, sessionid = 182

### Data Modeling
1. The combination of `User_id`, `Session_id`, and `Item_In_Session` is unique for each song played by a user within a session. Hence, they form a **composite primary key**.
2. `User_id` and `Session_id` are used as **partition keys** to distribute the data efficiently and ensure fast lookups for user-specific song play history.
3. `Item_In_Session` serves as the **clustering column**, allowing ordering of songs played in a session in **descending order**.

The required fields **artist name, song title, and user first and last names** can be retrieved using the following table structure:

```sql
CREATE TABLE IF NOT EXISTS user_song_played (
    User_id INT,
    Session_id INT,
    Item_In_Session INT,
    Artist TEXT,
    Song TEXT,
    First_name TEXT,
    Last_name TEXT,
    PRIMARY KEY ((User_id, Session_id), Item_In_Session))
    WITH CLUSTERING ORDER BY (Item_In_Session DESC);

```

In [11]:
# Structuring the tables based on the below query
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute("DROP TABLE IF EXISTS user_song_played") # Drop table if it already exist to avoid duplicates

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_song_played (
        User_id INT,
        Session_id INT,
        Item_In_Session INT,
        Artist TEXT,
        Song TEXT,
        First_name TEXT,
        Last_name TEXT,
        PRIMARY KEY ((User_id, Session_id),Item_In_Session)) WITH CLUSTERING ORDER BY (Item_In_Session DESC);
        
        """)

except Exception as e:
    print(e)

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements into the `query` variable
        query = "INSERT INTO user_song_played(User_id, Session_id,Item_In_Session, Artist, Song, First_name,Last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query,(int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [13]:
## Adding in the SELECT statement to verify the data was entered into the table
query = "select Artist, Song, First_name,Last_name, Item_In_Session from user_song_played where User_id=10 and Session_id=182  "
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows) # Convert to DataFrame
except Exception as e:
    print(e)
    
display(df)
    
    

,artist,song,first_name,last_name,item_in_session
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3
1,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
2,Three Drives,Greece 2000,Sylvie,Cruz,1
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0


## Query 3:

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Data Modeling
1. The combination of `Song` and `User_id` is unique for each user who listened to a particular song. Therefore, they form a **composite primary key**.
2. `Song` is used as the **partition key** to ensure that all users who listened to the same song are stored together, optimizing retrieval for song-based queries.
3. `User_id` is used as a **clustering column** to uniquely identify users and facilitate efficient searches.

The required fields **song title, user first name, and user last name** can be retrieved using the following table structure:

```sql
CREATE TABLE IF NOT EXISTS user_listened_song (
    Song TEXT,
    User_id INT,
    First_name TEXT,
    Last_name TEXT,
    PRIMARY KEY (Song, User_id));
```

In [14]:
# Structuring the tables based on the below query
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("DROP TABLE IF EXISTS user_listened_song") # Drop table if it already exist to avoid duplicates

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_listened_song (
        Song TEXT,
        User_id INT,
        First_name TEXT,
        Last_name TEXT,
        PRIMARY KEY (Song, User_id))
        
        """)

except Exception as e:
    print(e)

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements into the `query` variable
        query = "INSERT INTO user_listened_song(Song, User_id, First_name, Last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query,(line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)
            
            
            
            

In [16]:
## Adding in the SELECT statement to verify the data was entered into the table
query = "select First_name,Last_name from user_listened_song where Song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows._current_rows) # Convert to DataFrame
except Exception as e:
    print(e)
    
display(df)

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE IF EXISTS song_played")
session.execute("DROP TABLE IF EXISTS user_song_played")
session.execute("DROP TABLE IF EXISTS user_listened_song")


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()